## Deliverable 2. Create a Customer Travel Destinations Map.

In [55]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vallenar,CL,-28.5708,-70.7581,86.22,24,1,9.46,clear sky
1,1,Luwuk,ID,-0.9516,122.7875,77.79,87,100,1.83,moderate rain
2,2,Mana,GF,5.6592,-53.7767,80.87,73,95,11.21,overcast clouds
3,3,Souillac,MU,-20.5167,57.5167,79.18,83,40,0.00,scattered clouds
4,4,Sapa,PH,5.1544,120.3241,79.25,82,98,8.55,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vallenar,CL,-28.5708,-70.7581,86.22,24,1,9.46,clear sky
1,1,Luwuk,ID,-0.9516,122.7875,77.79,87,100,1.83,moderate rain
2,2,Mana,GF,5.6592,-53.7767,80.87,73,95,11.21,overcast clouds
3,3,Souillac,MU,-20.5167,57.5167,79.18,83,40,0.00,scattered clouds
4,4,Sapa,PH,5.1544,120.3241,79.25,82,98,8.55,overcast clouds
6,6,San Juan,AR,-31.5375,-68.5364,85.39,15,43,7.63,scattered clouds
9,9,Vaini,TO,-21.2000,-175.2000,77.16,94,40,6.91,light intensity shower rain
12,12,Rikitea,PF,-23.1203,-134.9692,78.87,70,3,11.48,clear sky
13,13,Castro,BR,-24.7911,-50.0119,77.83,55,3,7.14,clear sky
18,18,Puerto Ayora,EC,-0.7393,-90.3518,84.16,81,73,11.16,light rain


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                243
City                   243
Country                242
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)

preferred_cities_df.count()

City_ID                242
City                   242
Country                242
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vallenar,CL,86.22,clear sky,-28.5708,-70.7581,
1,Luwuk,ID,77.79,moderate rain,-0.9516,122.7875,
2,Mana,GF,80.87,overcast clouds,5.6592,-53.7767,
3,Souillac,MU,79.18,scattered clouds,-20.5167,57.5167,
4,Sapa,PH,79.25,overcast clouds,5.1544,120.3241,
6,San Juan,AR,85.39,scattered clouds,-31.5375,-68.5364,
9,Vaini,TO,77.16,light intensity shower rain,-21.2000,-175.2000,
12,Rikitea,PF,78.87,clear sky,-23.1203,-134.9692,
13,Castro,BR,77.83,clear sky,-24.7911,-50.0119,
18,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vallenar,CL,86.22,clear sky,-28.5708,-70.7581,Humacao Bed & Breakfast
1,Luwuk,ID,77.79,moderate rain,-0.9516,122.7875,Hotel Santika Luwuk
2,Mana,GF,80.87,overcast clouds,5.6592,-53.7767,LE SAMANA
3,Souillac,MU,79.18,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
4,Sapa,PH,79.25,overcast clouds,5.1544,120.3241,
6,San Juan,AR,85.39,scattered clouds,-31.5375,-68.5364,Hotel Provincial
9,Vaini,TO,77.16,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
12,Rikitea,PF,78.87,clear sky,-23.1203,-134.9692,People ThankYou
13,Castro,BR,77.83,clear sky,-24.7911,-50.0119,CHACARA BAILLY
18,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [68]:
# 7. Drop the rows where there is no Hotel Name.
#Replace empty strings with 'Nan'
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vallenar,CL,86.22,clear sky,-28.5708,-70.7581,Humacao Bed & Breakfast
1,Luwuk,ID,77.79,moderate rain,-0.9516,122.7875,Hotel Santika Luwuk
2,Mana,GF,80.87,overcast clouds,5.6592,-53.7767,LE SAMANA
3,Souillac,MU,79.18,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
6,San Juan,AR,85.39,scattered clouds,-31.5375,-68.5364,Hotel Provincial
9,Vaini,TO,77.16,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
12,Rikitea,PF,78.87,clear sky,-23.1203,-134.9692,People ThankYou
13,Castro,BR,77.83,clear sky,-24.7911,-50.0119,CHACARA BAILLY
18,Puerto Ayora,EC,84.16,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Antsohihy,MG,76.30,overcast clouds,-14.8796,47.9875,Hotel de Ville


In [70]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
current_description = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=0.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))